# Steamworks Data Visualization Dashboard

Interactive dashboard for analyzing game performance metrics from Steamworks data.

In [1]:
# Import required libraries
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from datetime import datetime, timedelta
import os
import sys

# Add lib directory to path - Jupyter notebook compatible
current_dir = os.getcwd()
lib_path = os.path.join(current_dir, 'lib')
if lib_path not in sys.path:
    sys.path.insert(0, lib_path)

# Import our custom modules
from data_loader import get_dau_new_users_trend, get_revenue_trend, get_json_data_for_pie_charts
from pie_charts import create_game_pie_charts_row
from chart_builder import GAME_COLORS, CHART_CONFIG

print("All modules imported successfully")


INFO:db_connector:Loaded environment variables from D:\Steamworks_Crawler\SteamWorks_crawler\Visualization\config\.env


All modules imported successfully


In [2]:
# Set date range (start from Oct 1st, 2025)
end_date = datetime.now().strftime('%Y-%m-%d')
start_date = '2025-10-01'  # Fixed start date as requested

print(f"Date Range: {start_date} to {end_date}")

# Load data
df_dau = get_dau_new_users_trend(start_date, end_date)
df_revenue = get_revenue_trend(start_date, end_date)
json_data = get_json_data_for_pie_charts(start_date, end_date)

print(f"Loaded {len(df_dau)} rows of DAU/New Users data")
print(f"Loaded {len(df_revenue)} rows of Revenue data")
print(f"Loaded JSON data for {len(json_data)} games")


INFO:data_loader:Fetching DAU and new users trend from 2025-10-01 to 2025-10-23
INFO:db_connector:Database engine created successfully for steamworks_crawler@localhost:3306
INFO:mysql.connector:package: mysql.connector.plugins
INFO:mysql.connector:plugin_name: caching_sha2_password
INFO:mysql.connector:AUTHENTICATION_PLUGIN_CLASS: MySQLCachingSHA2PasswordAuthPlugin
INFO:data_loader:Retrieved 88 rows of DAU/new users data
INFO:data_loader:Fetching revenue trend from 2025-10-01 to 2025-10-23
INFO:db_connector:Database engine created successfully for steamworks_crawler@localhost:3306
INFO:data_loader:Retrieved 88 rows of revenue data
INFO:db_connector:Database engine created successfully for steamworks_crawler@localhost:3306
INFO:data_loader:Loaded JSON data for 4 games


Date Range: 2025-10-01 to 2025-10-23
Loaded 88 rows of DAU/New Users data
Loaded 88 rows of Revenue data
Loaded JSON data for 4 games


## Interactive Game Trends

Interactive charts with game selection controls. Use the checkboxes below to toggle which games are displayed in each chart.

In [3]:
# Interactive Game Selection Controls
import ipywidgets as widgets
from IPython.display import display, clear_output

# Create checkboxes for each game (2x2 grid layout)
checkbox_container = widgets.GridBox(
    children=[
        widgets.Checkbox(
            value=True,
            description='Delta Force',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='200px')
        ),
        widgets.Checkbox(
            value=True,
            description='Arena Breakout: Infinite',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='200px')
        ),
        widgets.Checkbox(
            value=True,
            description='Road to Empress',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='200px')
        ),
        widgets.Checkbox(
            value=True,
            description='Terminull Brigade',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='200px')
        )
    ],
    layout=widgets.Layout(
        grid_template_columns='200px 200px',
        grid_template_rows='50px 50px',
        grid_gap='10px',
        width='420px'
    )
)

# Create output areas for charts
chart_output = widgets.Output()

# Function to update charts based on checkbox selection
def update_charts(change):
    with chart_output:
        clear_output(wait=True)
        
        # Get selected games
        selected_games = []
        for i, checkbox in enumerate(checkbox_container.children):
            if checkbox.value:
                game_names = ['Delta Force', 'Arena Breakout: Infinite', 'Road to Empress', 'Terminull Brigade']
                selected_games.append(game_names[i])
        
        if not selected_games:
            print("Please select at least one game to display.")
            return
        
        # Create subplot with 3 charts side by side
        fig = make_subplots(
            rows=1, cols=3,
            subplot_titles=("New Players", "DAU", "Revenue"),
            horizontal_spacing=0.08
        )
        
        # Add traces for each selected game
        for game in selected_games:
            if game in df_dau['game_name'].values:
                game_data = df_dau[df_dau['game_name'] == game].sort_values('stat_date')
                
                # New Players chart (column 1)
                fig.add_trace(
                    go.Scatter(
                        x=game_data['stat_date'],
                        y=game_data['new_players'],
                        mode='lines+markers',
                        name=game,
                        line=dict(color=GAME_COLORS[game], width=3),
                        marker=dict(size=6),
                        hovertemplate=f'<b>{game}</b><br>Date: %{{x}}<br>New Players: %{{y:,.0f}}<extra></extra>',
                        showlegend=True
                    ),
                    row=1, col=1
                )
                
                # DAU chart (column 2)
                fig.add_trace(
                    go.Scatter(
                        x=game_data['stat_date'],
                        y=game_data['dau'],
                        mode='lines+markers',
                        name=game,
                        line=dict(color=GAME_COLORS[game], width=3),
                        marker=dict(size=6),
                        hovertemplate=f'<b>{game}</b><br>Date: %{{x}}<br>DAU: %{{y:,.0f}}<extra></extra>',
                        showlegend=False
                    ),
                    row=1, col=2
                )
        
        # Add revenue traces
        for game in selected_games:
            if game in df_revenue['game_name'].values:
                game_data = df_revenue[df_revenue['game_name'] == game].sort_values('stat_date')
                
                # Revenue chart (column 3)
                fig.add_trace(
                    go.Scatter(
                        x=game_data['stat_date'],
                        y=game_data['daily_total_revenue'],
                        mode='lines+markers',
                        name=game,
                        line=dict(color=GAME_COLORS[game], width=3),
                        marker=dict(size=6),
                        hovertemplate=f'<b>{game}</b><br>Date: %{{x}}<br>Revenue: $%{{y:,.0f}}<extra></extra>',
                        showlegend=False
                    ),
                    row=1, col=3
                )
        
        # Update layout
        fig.update_layout(
            height=600,
            width=3600,  # 3 charts * 1200px each
            template='plotly_white',
            font=dict(family='Arial', size=12),
            showlegend=True,
            legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
        )
        
        # Update axes
        fig.update_xaxes(title_text="Date", row=1, col=1)
        fig.update_xaxes(title_text="Date", row=1, col=2)
        fig.update_xaxes(title_text="Date", row=1, col=3)
        
        fig.update_yaxes(title_text="New Players", row=1, col=1)
        fig.update_yaxes(title_text="Daily Active Users", row=1, col=2)
        fig.update_yaxes(title_text="Daily Revenue ($)", row=1, col=3)
        
        fig.show()

# Attach the update function to all checkboxes
for checkbox in checkbox_container.children:
    checkbox.observe(update_charts, names='value')

# Display the controls and initial charts
print("Game Selection Controls:")
display(checkbox_container)
print("\nInteractive Charts:")
display(chart_output)

# Trigger initial chart display
update_charts(None)

Game Selection Controls:


GridBox(children=(Checkbox(value=True, description='Delta Force', layout=Layout(width='200px'), style=Checkbox…


Interactive Charts:


Output()

## Game Breakdown Analysis

Pie charts showing geographic and platform breakdowns for each game. Each row represents one game with 5 pie charts:
- **Country DAU**: Daily Active Users by country
- **Country Downloads**: Downloads by country  
- **Region Downloads**: Downloads by region
- **Country Revenue**: Revenue by country
- **Region Revenue**: Revenue by region

Each pie chart shows the top 3 items plus "Others" (remaining 7 items combined).

In [4]:
# Create pie chart output area
pie_chart_output = widgets.Output()

# Function to update pie charts based on checkbox selection
def update_pie_charts(change):
    with pie_chart_output:
        clear_output(wait=True)
        
        # Get selected games
        selected_games = []
        for i, checkbox in enumerate(checkbox_container.children):
            if checkbox.value:
                game_names = ['Delta Force', 'Arena Breakout: Infinite', 'Road to Empress', 'Terminull Brigade']
                selected_games.append(game_names[i])
        
        if not selected_games:
            print("Please select at least one game to display.")
            return
        
        # Create pie charts for each selected game
        for game in selected_games:
            if game in json_data:
                game_data = json_data[game]
                fig = create_game_pie_charts_row(game_data, game)
                fig.show()
            else:
                print(f"No data available for {game}")

# Attach the update function to all checkboxes
for checkbox in checkbox_container.children:
    checkbox.observe(update_pie_charts, names='value')

# Display the pie charts
print("Game Breakdown Analysis:")
display(pie_chart_output)

# Trigger initial pie chart display
update_pie_charts(None)

Game Breakdown Analysis:


Output()

## Chart Export Information

To export charts as PNG images or HTML files, use the `export_charts.py` script:

```bash
python export_charts.py
```

This will create static versions of all charts in the `exports/charts/` directory.